<a href="https://colab.research.google.com/github/desaiankitb/pytorch-basics/blob/main/tutorial-pytorch-org/00_5_Optimization_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline 

# Optimizing the model parameters

- Now that we have a model and data it's time to train, validate and test our model by optimizing its parameters on our data. Training a model is an iterative process; in each iteration (called an *epoch*) the model makes a guess about the output, calculates the error in its guess (*loss*), collects the derivatives of the error with respect to its parameters (as we saw in the module), and **optimizes** these parameters using gradient descent. For a more detailed walkthrough of this process, checkout this video on [backpropogation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

- We load the code from the previous module on **Datasets & DataLoaders** and **Build Model**

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader 
from torchvision import datasets 
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_datasetloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(), 
        nn.Linear(512, 10),
        nn.ReLU()
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork()


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Setting hyperparameters

- Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperpameter values can impact model training and coverage rates ([read more](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html) about hyperparameter tuning)

- We define the following hyperparameters for training: 
  - **Number of Epochs** - the number times to iterate over dataset
  - **Batch Size** - the number of data samples seen by the model in each epoch 
  - **Learning Rate** - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behaviour during training. 

In [3]:
learning_rate = 1e-3
batch_size = 64
epoch = 5

## Add an optimization loop

- Once we set our hyperparameters, we can then train and optimize our model with an opotimization loop. Each iteration of the optimization loop is called an **epoch**. 

- Each epoch consists of two main parts: 

  - **The Train Loop** - iterate over the training dataset and try to converge to optimal parameters. 
  - ** The Validation/Test Loop** - iterate over the test dataset to check if model performance is improving. 

- Let's briefly familiarize overselves with some of the concepts used in the training loop. Jump ahead to see the `full-impl-label` of the optimization loop. 

## Add a loss function 

- When presented with some training data, our untrained network is likely to give the correct answer. **Loss Function** measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value. 

- Common loss functions include [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss) (Mean Square Error) for regression tasks, and [nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss) (Negative Log Likelihood) for classification. [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) combines `nn.LogSoftmax` and `nn.NLLLoss`. 

- We pass our model's output logits to `nn.CrossEntropyLoss`, which will normalize the logits and compute the prediction error. 

In [4]:
# Initialize the loss function 
loss_fn = nn.CrossEntropyLoss()